# ASSOCIATION RULE LEARNING

In [38]:
#!pip install mlxtend
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.expand_frame_repr', False)
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from mlxtend.frequent_patterns import apriori, association_rules

def load_retail():
    df_= pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")
    return df_
df_ = load_retail()
df = df_.copy()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


In [39]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    #dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [40]:
def crm_data_prep(dataframe):
    dataframe.dropna(axis=0, inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    return dataframe

In [41]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    
check_df(df)

##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity         InvoiceDate   Price  Customer ID         Country
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00 2.55000  17850.00000  United Kingdom
1  536365     71053                 WHITE METAL LANTERN         6 2010-12-01 08:26:00 3.39000  17850.00000  United Kingdom
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00 2.75000  17850.00000  United Kingdom
##################### Tail #####################
       Invoice StockCode                    

In [42]:
df_prep = crm_data_prep(df)
check_df(df_prep)

##################### Shape #####################
(397925, 9)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity         InvoiceDate   Price  Customer ID         Country  TotalPrice
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER   6.00000 2010-12-01 08:26:00 2.55000  17850.00000  United Kingdom    15.30000
1  536365     71053                 WHITE METAL LANTERN   6.00000 2010-12-01 08:26:00 3.39000  17850.00000  United Kingdom    20.34000
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER   8.00000 2010-12-01 08:26:00 2.75000  17850.00000  United Kingdom    22.00000
###############

In [43]:
# CLTV Prediction

def create_cltv_p(dataframe):
    today_date = dt.datetime(2011, 12, 11)

    ## Recency value special for each customer.
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days,
                                                                lambda date: (today_date - date.min()).days],
                                                'Invoice': lambda num: num.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

    rfm.columns = rfm.columns.droplevel(0)

    ## recency_cltv_p
    rfm.columns = ['recency_cltv_p', 'T', 'frequency', 'monetary']

    ## simple monetary_avg
    rfm["monetary"] = rfm["monetary"] / rfm["frequency"]

    rfm.rename(columns={"monetary": "monetary_avg"}, inplace=True)


    # Calculation of WEEKLY RECENCY and WEEKLY T for BGNBD Model
    ## recency_weekly_cltv_p
    rfm["recency_weekly_cltv_p"] = rfm["recency_cltv_p"] / 7
    rfm["T_weekly"] = rfm["T"] / 7

    # Control
    rfm = rfm[rfm["monetary_avg"] > 0]

    ## We want to customer who has purchased more than one
    rfm = rfm[(rfm['frequency'] > 1)]

    rfm["frequency"] = rfm["frequency"].astype(int)

    # BGNBD
    bgf = BetaGeoFitter(penalizer_coef=0.01)
    bgf.fit(rfm['frequency'],
            rfm['recency_weekly_cltv_p'],
            rfm['T_weekly'])

    # exp_sales_1_month
    rfm["exp_sales_1_month"] = bgf.predict(4,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])
    # exp_sales_3_month
    rfm["exp_sales_3_month"] = bgf.predict(12,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])

    # Gamma Gamma
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(rfm['frequency'], rfm['monetary_avg'])
    rfm["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm['frequency'],
                                                                             rfm['monetary_avg'])
    # 6 months cltv_p
    cltv = ggf.customer_lifetime_value(bgf,
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'],
                                       rfm['monetary_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)

    rfm["cltv_p"] = cltv

    # minmaxscaler
    scaler = MinMaxScaler(feature_range=(1, 100))
    scaler.fit(rfm[["cltv_p"]])
    rfm["cltv_p"] = scaler.transform(rfm[["cltv_p"]])

    # cltv_p_segment (3 segments)
    rfm["cltv_p_segment"] = pd.qcut(rfm["cltv_p"], 3, labels=["C", "B", "A"])

    ## recency_cltv_p, recency_weekly_cltv_p
    rfm = rfm[["recency_cltv_p", "T", "monetary_avg", "recency_weekly_cltv_p", "T_weekly",
               "exp_sales_1_month", "exp_sales_3_month", "expected_average_profit",
               "cltv_p", "cltv_p_segment"]]


    return rfm


cltv_p = create_cltv_p(df_prep)
cltv_p.head()

,recency_cltv_p,T,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,
12347.00000,365,368,615.71429,52.14286,52.57143,0.56168,1.67807,631.91197,2.93352,A
12348.00000,282,359,442.69500,40.28571,51.28571,0.36432,1.08795,463.74554,1.91939,B
12352.00000,260,297,219.54250,37.14286,42.42857,0.73916,2.20686,224.88667,1.90415,B
12356.00000,302,326,937.14333,43.14286,46.57143,0.33304,0.99393,995.99768,2.80249,A
12358.00000,149,151,575.21000,21.28571,21.57143,0.47405,1.40896,631.90095,2.61298,A


In [44]:
######################################
# Create the data set according to the user ids of the 3 segments.
######################################

# id
a_segment_ids = cltv_p[cltv_p["cltv_p_segment"] == "A"].index
b_segment_ids = cltv_p[cltv_p["cltv_p_segment"] == "B"].index
c_segment_ids = cltv_p[cltv_p["cltv_p_segment"] == "C"].index

# Creation of df's according to these 3 ids
a_segment_df = df_prep[df_prep["Customer ID"].isin(a_segment_ids)]
b_segment_df = df_prep[df_prep["Customer ID"].isin(b_segment_ids)]
c_segment_df = df_prep[df_prep["Customer ID"].isin(c_segment_ids)]
a_segment_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
9,536368,22960,JAM MAKING SET WITH JARS,6.00000,2010-12-01 08:34:00,4.25000,13047.00000,United Kingdom,25.50000
10,536368,22913,RED COAT RACK PARIS FASHION,3.00000,2010-12-01 08:34:00,4.95000,13047.00000,United Kingdom,14.85000
11,536368,22912,YELLOW COAT RACK PARIS FASHION,3.00000,2010-12-01 08:34:00,4.95000,13047.00000,United Kingdom,14.85000
12,536368,22914,BLUE COAT RACK PARIS FASHION,3.00000,2010-12-01 08:34:00,4.95000,13047.00000,United Kingdom,14.85000
13,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32.00000,2010-12-01 08:34:00,1.69000,13047.00000,United Kingdom,54.08000


In [45]:
# Create association rules for each segment
def create_invoice_product_df(dataframe):
    return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
        applymap(lambda x: 1 if x > 0 else 0)


def create_rules(dataframe, country=False, head=5):
    if country:
        dataframe = dataframe[dataframe['Country'] == country]
        dataframe = create_invoice_product_df(dataframe)
        frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True, low_memory=True)
        rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
        print(rules.sort_values("lift", ascending=False).head(head))
    else:
        dataframe = create_invoice_product_df(dataframe)
        frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True, low_memory=True)
        rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
        print(rules.sort_values("lift", ascending=False).head(head))

    return rules

rules_a = create_rules(a_segment_df)
rules_a.head()

                antecedents             consequents  antecedent support  consequent support  support  confidence     lift  leverage  conviction
287     (HERB MARKER THYME)     (HERB MARKER BASIL)             0.01081             0.01113  0.01018     0.94175 84.62402   0.01006    16.97563
286     (HERB MARKER BASIL)     (HERB MARKER THYME)             0.01113             0.01081  0.01018     0.91509 84.62402   0.01006    11.65042
293  (HERB MARKER ROSEMARY)   (HERB MARKER PARSLEY)             0.01155             0.01081  0.01050     0.90909 84.06884   0.01037    10.88105
292   (HERB MARKER PARSLEY)  (HERB MARKER ROSEMARY)             0.01081             0.01155  0.01050     0.97087 84.06884   0.01037    33.93683
285  (HERB MARKER ROSEMARY)     (HERB MARKER BASIL)             0.01155             0.01113  0.01071     0.92727 83.32333   0.01058    13.59698


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(REGENCY CAKESTAND 3 TIER),(6 RIBBONS RUSTIC CHARM),0.10257,0.04735,0.01102,0.10747,2.26978,0.00617,1.06736
1,(6 RIBBONS RUSTIC CHARM),(REGENCY CAKESTAND 3 TIER),0.04735,0.10257,0.01102,0.23282,2.26978,0.00617,1.16977
2,(SCANDINAVIAN REDS RIBBONS),(6 RIBBONS RUSTIC CHARM),0.02100,0.04735,0.01039,0.49500,10.45427,0.00940,1.88644
3,(6 RIBBONS RUSTIC CHARM),(SCANDINAVIAN REDS RIBBONS),0.04735,0.02100,0.01039,0.21951,10.45427,0.00940,1.25435
4,(60 CAKE CASES DOLLY GIRL DESIGN),(PACK OF 60 SPACEBOY CAKE CASES),0.02163,0.02478,0.01050,0.48544,19.59232,0.00996,1.89524


In [46]:
print(rules_a["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0])
print(rules_b["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0])
print(rules_c["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0])

6 RIBBONS RUSTIC CHARM
FELTCRAFT 6 FLOWER FRIENDS
60 TEATIME FAIRY CAKE CASES


In [47]:
rules_a = create_rules(a_segment_df)
product_a = rules_a["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0]

rules_b = create_rules(b_segment_df)
product_b = rules_b["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0]

rules_c = create_rules(c_segment_df)
product_c = rules_c["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0]

                antecedents             consequents  antecedent support  consequent support  support  confidence     lift  leverage  conviction
287     (HERB MARKER THYME)     (HERB MARKER BASIL)             0.01081             0.01113  0.01018     0.94175 84.62402   0.01006    16.97563
286     (HERB MARKER BASIL)     (HERB MARKER THYME)             0.01113             0.01081  0.01018     0.91509 84.62402   0.01006    11.65042
293  (HERB MARKER ROSEMARY)   (HERB MARKER PARSLEY)             0.01155             0.01081  0.01050     0.90909 84.06884   0.01037    10.88105
292   (HERB MARKER PARSLEY)  (HERB MARKER ROSEMARY)             0.01081             0.01155  0.01050     0.97087 84.06884   0.01037    33.93683
285  (HERB MARKER ROSEMARY)     (HERB MARKER BASIL)             0.01155             0.01113  0.01071     0.92727 83.32333   0.01058    13.59698
                antecedents             consequents  antecedent support  consequent support  support  confidence     lift  leverage  con

In [48]:
# Control
def check_id(stock_code):
    product_name = df_prep[df_prep["StockCode"] == stock_code][["Description"]].values[0].tolist()
    return print(product_name)

check_id(20719)

['WOODLAND CHARLOTTE BAG']


In [49]:
# We want to recommend for only Germany people
germany_ids = df_prep[df_prep["Country"] == "Germany"]["Customer ID"].drop_duplicates()

cltv_p["recommended_product"] = ""

cltv_p.loc[cltv_p.index.isin(germany_ids)]

cltv_p.loc[(cltv_p.index.isin(germany_ids)) & (cltv_p["cltv_p_segment"] == "A")]

cltv_p.loc[(cltv_p.index.isin(germany_ids)) & (cltv_p["cltv_p_segment"] == "A"), "recommended_product"] = product_a

cltv_p.loc[(cltv_p.index.isin(germany_ids)) & (cltv_p["cltv_p_segment"] == "A")]

cltv_p.loc[(cltv_p.index.isin(germany_ids)) & (cltv_p["cltv_p_segment"] == "B"), "recommended_product"] = product_b
cltv_p.loc[(cltv_p.index.isin(germany_ids)) & (cltv_p["cltv_p_segment"] == "B")]

cltv_p.loc[(cltv_p.index.isin(germany_ids)) & (cltv_p["cltv_p_segment"] == "C"), "recommended_product"] = product_c
cltv_p.loc[(cltv_p.index.isin(germany_ids)) & (cltv_p["cltv_p_segment"] == "C")]

,recency_cltv_p,T,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment,recommended_product
Customer ID,,,,,,,,,,,
12427.00000,349,372,275.26667,49.85714,53.14286,0.29840,0.89100,293.48575,1.47644,C,60 TEATIME FAIRY CAKE CASES
12468.00000,128,317,364.77000,18.28571,45.28571,0.20220,0.60313,401.46758,1.44056,C,60 TEATIME FAIRY CAKE CASES
12492.00000,42,106,131.60500,6.00000,15.14286,0.52872,1.56846,146.15016,1.41445,C,60 TEATIME FAIRY CAKE CASES
12522.00000,12,52,96.36000,1.71429,7.42857,0.78964,2.33387,107.55663,1.45223,C,60 TEATIME FAIRY CAKE CASES
12527.00000,243,326,116.23667,34.71429,46.57143,0.32248,0.96243,124.69226,1.21851,C,60 TEATIME FAIRY CAKE CASES
12559.00000,8,320,280.85500,1.14286,45.71429,0.01722,0.05136,309.58003,1.02893,C,60 TEATIME FAIRY CAKE CASES
12592.00000,121,214,218.80000,17.28571,30.57143,0.33496,0.99733,241.62934,1.43749,C,60 TEATIME FAIRY CAKE CASES
12601.00000,142,333,114.83400,20.28571,47.57143,0.24146,0.72098,119.73532,1.15729,C,60 TEATIME FAIRY CAKE CASES
12649.00000,105,213,207.83667,15.00000,30.42857,0.40035,1.19265,221.91595,1.48080,C,60 TEATIME FAIRY CAKE CASES


In [50]:
cltv_p.loc[cltv_p.index.isin(germany_ids)].head(20)

,recency_cltv_p,T,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment,recommended_product
Customer ID,,,,,,,,,,,
12427.00000,349,372,275.26667,49.85714,53.14286,0.29840,0.89100,293.48575,1.47644,C,60 TEATIME FAIRY CAKE CASES
12468.00000,128,317,364.77000,18.28571,45.28571,0.20220,0.60313,401.46758,1.44056,C,60 TEATIME FAIRY CAKE CASES
12471.00000,362,365,647.27233,51.71429,52.14286,2.05998,6.15703,651.16315,8.31495,A,6 RIBBONS RUSTIC CHARM
12472.00000,336,370,938.87286,48.00000,52.85714,0.55463,1.65704,963.28521,3.91057,A,6 RIBBONS RUSTIC CHARM
12473.00000,162,192,606.49400,23.14286,27.42857,0.71680,2.13561,629.07221,3.44095,A,6 RIBBONS RUSTIC CHARM
12474.00000,312,330,409.76611,44.57143,47.14286,1.38754,4.14548,413.96409,4.12924,A,6 RIBBONS RUSTIC CHARM
12476.00000,345,362,619.67455,49.28571,51.71429,0.82890,2.47679,629.95123,3.84559,A,6 RIBBONS RUSTIC CHARM
12477.00000,278,302,2203.29000,39.71429,43.14286,0.58262,1.73929,2269.55010,8.19023,A,6 RIBBONS RUSTIC CHARM
12480.00000,307,337,820.40750,43.85714,48.14286,0.39292,1.17310,858.58710,2.83484,A,6 RIBBONS RUSTIC CHARM
